# Stable Diffusion WebUI 带lora训练

## 阿里云 DSW 一键脚本 By bilibili@十字鱼
### 基于bilibili@秋葉aaaki大佬的云端包修改并整合

#### 简介

- 十字鱼 https://space.bilibili.com/893892
- 更新日期 05.12

## 1 安装 第一次运行时执行

### 1.1 SD安装

In [ ]:
!nvidia-smi
%cd /mnt/workspace
!apt update
!apt install -y aria2
!git clone https://gitcode.net/overbill1683/stable-diffusion-webui
%cd stable-diffusion-webui
!mkdir repositories
%cd repositories
!git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
!git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
!git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
!git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
!git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"
%cd /mnt/workspace/stable-diffusion-webui
!wget -O "config.json" "https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/config.json"
!wget -O "styles.csv" "https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/styles.csv"
# 安装常用插件
extension_urls = """
https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete
https://gitcode.net/overbill1683/stable-diffusion-webui-localization-zh_Hans
https://gitcode.net/ranting8323/sd-webui-additional-networks
https://ghproxy.com/https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git
https://gitcode.net/ranting8323/stable-diffusion-webui-wd14-tagger
https://gitcode.net/ranting8323/sd-webui-controlnet
"""
extensions = extension_urls.strip().split("\n")
%cd /mnt/workspace/stable-diffusion-webui
for e in extensions:
    !git -C "extensions" clone {e}

### 1.2 lora训练安装
- 不训练可跳过
- ./lora-scripts/sd-scripts文件夹里有文件才成功

In [ ]:
import os
!python -m pip install --upgrade pip
%cd /mnt/workspace
!git clone --recurse-submodules https://ghproxy.com/https://github.com/Akegarasu/lora-scripts
if os.path.exists('lora-scripts'):
    %cd lora-scripts
    !mkdir train
    %cd train
    !mkdir 10_glut
    %cd ../
    !git reset --hard
    !git pull
    !git submodule init
    !git submodule update
    !wget -O "train.sh" "https://ghproxy.com/https://github.com/gluttony-10/lora-scripts/blob/main/train.sh"
    !pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 -f https://mirror.sjtu.edu.cn/pytorch-wheels/torch_stable.html -i https://mirrors.bfsu.edu.cn/pypi/web/simple
    !pip install -U -I --no-deps xformers==0.0.17 -i https://mirrors.bfsu.edu.cn/pypi/web/simple
    if os.path.exists('sd-scripts'):
        %cd sd-scripts
        !pip install --upgrade -r requirements.txt -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade lion-pytorch dadaptation -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade lycoris-lora -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade fastapi uvicorn -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !sudo apt-get install -y nvidia-cuda-toolkit

## 2 下载模型和VAE
- 可修改根目录下model.txt文件来添加模型和其他文件

In [ ]:
%cd /mnt/workspace/
!wget -O "model.txt" "https://ghproxy.com/https://github.com/gluttony-10/1/blob/main/model.txt"
!aria2c --console-log-level=error -c -x 16 -s 16 -j 4 -i "model.txt"

## 3 训练lora
- 不训练可跳过
- 训练文件放进./lora-scripts/train/10_glut中
- 训练参数请修改 ./lora-scripts/train.sh

In [ ]:
%cd /mnt/workspace/lora-scripts
!bash train.sh

## 4 启动WebUI

In [ ]:
import os
package_envs  = [
      {
        "env": "GFPGAN_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379"
      },
      {
        "env": "CLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1"
      },
      {
        "env": "OPENCLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b"
      }
]
os.environ["PIP_INDEX_URL"] = "https://mirrors.bfsu.edu.cn/pypi/web/simple"
for i in package_envs:
    os.environ[i["env"]] = i["url"]

%cd /mnt/workspace/stable-diffusion-webui
!python launch.py --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access

/mnt/workspace/stable-diffusion-webui
Python 3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]
Commit hash: 5ab7f213bec2f816f9c5644becb32eb72c8ffb89
Installing requirements


Launching Web UI with arguments: --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access
2023-05-12 10:54:23.901029: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 10:54:24.113956: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-12 10:54:24.891531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: lib

## 5 压缩outputs文件夹
- 运行前先停止步骤5

In [ ]:
%cd /mnt/workspace/stable-diffusion-webui
!tar -zcvf 1.tar.gz outputs